In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_for_llm.csv to data_for_llm.csv


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('data_for_llm.csv')
df=df[['recipe_name','formatted_directions']]

# Prepare data for training
data = []
for index, row in df.iterrows():
    data.append({'input_text': f"Recipe: {row['recipe_name']}\nInstructions: {row['formatted_directions']}", 'target_text': row['formatted_directions']})


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))

# Tokenize and prepare data
def tokenize_function(examples):
    return tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)

# Tokenize the dataset
train_encodings = tokenize_function({'input_text': [x['input_text'] for x in data]})
train_labels = tokenize_function({'input_text': [x['target_text'] for x in data]})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
import torch

class RecipeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.labels['input_ids'])

train_dataset = RecipeDataset(train_encodings, train_labels)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()


Step,Training Loss
500,3.762800
1000,1.779100
1500,1.768900
2000,1.687900
2500,1.661100
3000,1.679700
3500,1.656300
4000,1.583600
4500,1.627000
5000,1.646700


TrainOutput(global_step=10000, training_loss=1.747525616455078, metrics={'train_runtime': 4143.9057, 'train_samples_per_second': 4.826, 'train_steps_per_second': 2.413, 'total_flos': 5225840640000000.0, 'train_loss': 1.747525616455078, 'epoch': 1.0})

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Save the model and tokenizer
model_dir = '/content/fine-tuned-recipe-model'
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)


('/content/fine-tuned-recipe-model/tokenizer_config.json',
 '/content/fine-tuned-recipe-model/special_tokens_map.json',
 '/content/fine-tuned-recipe-model/vocab.json',
 '/content/fine-tuned-recipe-model/merges.txt',
 '/content/fine-tuned-recipe-model/added_tokens.json')

In [ ]:
from google.colab import files
import shutil

# Zip the directory
shutil.make_archive(model_dir, 'zip', model_dir)

# Download the zip file
files.download(f'{model_dir}.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.save_pretrained('./fine-tuned-recipe-model')
tokenizer.save_pretrained('./fine-tuned-recipe-model')

from transformers import pipeline

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained('./fine-tuned-recipe-model')
tokenizer = GPT2Tokenizer.from_pretrained('./fine-tuned-recipe-model')

# Create a pipeline for text generation
recipe_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Example query
query = " who  is the president of america?"

# Generate a response
response = recipe_pipeline(query, max_length=150, num_return_sequences=1)

print(response[0]['generated_text'])


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


 who  is the president of america?

25 m m ovenComb, oniono, a, pepper a a large until powder 1 in. well., and heat/. a. until in, and; and 1..,. to the in to onion 1 about and;.2 and. and the is. garlic; in,. with,, mixture. 3 Add into are, to, in C baking baking and degrees) in of oven on the minutes the))
 over the and 2 minutes and on, in over
